In [31]:
from PIL import Image, ImageDraw, ImageFont
from annotator import text_label
import random

class Draw_Table():
    '''
    This class is used to print tables on invoices
    The table always has a 'Description' column as the first column and 'Brutto' as the last column.
    The 'Quantity', 'Tax %' and 'Netto' columns are optional. The number of rows is between 3 and 10.
    The user has to define the bounding box of the table and the currency.
    '''
    def __init__(self) -> None:
        # Possible column names
        self.column_names = ['Description', 'Quantity', 'Tax %', 'Netto', 'Brutto']
        # List of possible items in the table
        self.items = [
            'Electricity',
            'Water',
            'Gas',
            'Printer',
            'Maintenance fee',
            'Desktop monitor',
            'Designing cost',
            'Office chair',
            'Cell phone',
            'Laptop',
            'Network access point',
            'Network switch',
            'Professional beamer',
            'Smart white board',
            'Thermostat',
            'Office plants',
            'Professional software',
            'Printer ink',
            'Cleaning material',
            'Keyboard',
            'Network cable',
            'Security camera',
            'Carpet',
            'Washing machine',
            'Kitchen sink',
            'Garbage can'
        ]

    def __call__(self, drawer, bbox, currency):
        self.draw = ImageDraw.Draw(drawer)
        self.bbox = bbox                            # Bounding box of the table [x1, y1, x2, y2]
        self.currency = currency                    # CUrrency to use in the table
        self.font = ImageFont.truetype("arial.ttf", 30)
        self.num_items = random.randint(3, 10)      # How many rows does the table have (number of items)
        self.num_cols = random.randint(2, 5)        # How many columns does the table have
        self.increment = 50                         # Height of a row in pixels
        self.margin = 12                            # Distance between the table lines and text
        self.labels = []                            # Store the labels of the words

        self.draw_horizontal_lines()
        self.draw_vertical_lines()
        self.draw_content()

        return self.labels

    def get_x_positions(self):
        '''
        Calculates the horizontal positions where the table is getting divided (column boundaries)
        Only the table width is defined, the column positions depend on the number of the columns

        Return
            List of column boundaries
        '''
        # Width of the table x2 - x1
        full_width = self.bbox[2] - self.bbox[0]
        # Width of one column
        col_width = int(full_width / self.num_cols)
        # X coordinate of the beginning of the columns
        col_positions = []
        # The table has an offset, add this offset to the columns too
        # (The table doesn't begins at the very left of the invoice)
        pos = self.bbox[0]

        for _ in range(self.num_cols):
            col_positions.append(pos)
            pos += col_width
        col_positions.append(pos)

        return col_positions

    def draw_horizontal_lines(self):
        '''
        Draws the horizontal lines of the table
        '''
        # Get the XY positions of the first horizontal line
        left_x, left_y, right_x, right_y = self.bbox
        right_y = self.bbox[1]

        for _ in range(self.num_items):
            # Increase the Y value by the defined row height
            left_y += self.increment
            right_y += self.increment
            self.draw.line([(left_x, left_y), (right_x, right_y)], fill ="black", width = 2)

    def draw_vertical_lines(self):
        '''
        Draws the vertical lines of the table
        '''
        # Get the Y values of the vertical lines
        top_y = self.bbox[1] + self.increment
        bot_y = self.bbox[1] + (self.increment * self.num_items)

        # Get the X values of the vertical lines
        positions = self.get_x_positions()

        # Draw the lines one-by-one
        for pos_x in positions:  
            self.draw.line([(pos_x, top_y), (pos_x, bot_y)], fill ="black", width = 2)

    def draw_header(self):
        '''
        Draw the header of the table 
        The header is always bold and above the table
        '''
        # Get the X coordinate of the beginning of the columns
        positions = self.get_x_positions()
        
        # Print the first column header (description) to the left
        self.draw.text((positions[0] + self.margin, self.bbox[1]), self.column_names[0], fill='black', font=self.font, anchor='lm', stroke_width=1)

        # Print the rest of the columns to the right
        for col in range(self.num_cols - 2):
            self.draw.text((positions[col+2] - self.margin, self.bbox[1]), self.column_names[col+1], fill='black', font=self.font, anchor='rm', stroke_width=1)

        # Print the last column header (brutto) to the right
        self.draw.text((positions[-1] - self.margin, self.bbox[1]), self.column_names[-1], fill='black', font=self.font, anchor='rm', stroke_width=1)

    def draw_content(self):
        '''
        Populates the table with content
        '''
        # Draw the header part
        self.draw_header()

        # Sample random fake data
        descriptions = random.sample(self.items, self.num_items-1)  # Random items from the predefined list
        quantities = random.sample(range(1, 20), self.num_items-1)  # Random quantity between 1 and 20
        taxes = random.sample(range(5, 20), self.num_items-1)       # Tax rate between 5 and 20
        nettos = random.sample(range(5000), self.num_items-1)       # Netto price
        bruttos = [round(a+a*b*0.01, 1) for a,b in zip(nettos,taxes)]   # Brutto price based on netto and tax (rounded to 2 decimals)
        
        # Organize the generated data in a list (already a 2d table format)
        data = [descriptions, quantities, taxes, nettos, bruttos]

        # Get the X coordinate of the beginning of the columns
        positions = self.get_x_positions()

        # Print the data into the table
        for row in range(self.num_items -1):
            for col in range(self.num_cols -1):
                self.draw.text((positions[col +1] - self.margin, self.bbox[1]+row*self.increment+self.increment+self.margin), str(data[col][row]), fill='black', font=self.font, anchor='rt')
            # Print the brutto separately (currency has to be added)
            self.draw.text((positions[-1] - self.margin, self.bbox[1]+row*self.increment+self.increment+self.margin), str(data[-1][row])+' '+self.currency, fill='black', font=self.font, anchor='rt')

        # Print the sum field
        summa = sum(bruttos)
        summa = round(summa, 2)
        self.draw_sum_field(str(summa))

    def draw_sum_field(self, summa):
        x = self.bbox[2] - self.margin
        self.draw.text((x, self.bbox[1] + self.increment*self.num_items + self.margin), summa + ' ' + self.currency, fill='black', font=self.font, anchor='rt')
        text_width, text_height = self.draw.textsize(summa + ' ' + self.currency, font=self.font)
        self.draw.text((x - text_width - 8, self.bbox[1] + self.increment*self.num_items + self.margin), 'Sum: ', fill='black', font=self.font, anchor='rt', stroke_width=1)

In [4]:
class LayoutManager():
    def __init__(self) -> None:
        pass

    def __call__(self):
        layout = {
            'R_field': [100, 100, 600, 500],    # Recipient
            'S_field': [900, 350, 1538, 850],   # Supplier
            'L_field': [1000, 50, 1538, 200],   # Logo
            'T_field': [100, 1000, 1538, 1600], # Table
            'I_field': [100, 2000, 1538, 2200], # Information
            'Q_field': [550, 550, 800, 800],    # QR code
            'X_field': [100, 1650, 1538, 1900]  # Text field
        }

        return layout

In [ ]:
class Draw_customer_field():
    def __init__(self) -> None:
        pass

    def __call__(self):
        pass

In [32]:
from fakedata import FakeData

class InvoiceGenerator():
    def __init__(self, canvas, num_documents = 1, det = True) -> None:
        self.canvas = canvas
        self.drawer = ImageDraw.Draw(self.canvas)
        self.font = ImageFont.truetype("arial.ttf", 30)
        self.num_documents = num_documents
        self.det = det        

    def __call__(self):
        # 1. Generate the layout
        layout_manager = LayoutManager()
        self.layout = layout_manager()
        # self.visualize_layout()

        # 2. Generate the fake data
        fake_data_generator = FakeData()
        fake_data = fake_data_generator(self.num_documents, self.det)

        # 3. Draw the fake data on the image
        table_drawer = Draw_Table()
        for document in fake_data:
            # Draw the table on the document
            table_drawer(self.canvas, self.layout['T_field'], document['I_Currency'])
            self.canvas.show()

    def visualize_layout(self):
        for key in self.layout:
            self.drawer.rectangle(self.layout[key], outline='black')
            self.drawer.text(self.layout[key], key, font=self.font, fill='black')
        self.canvas.show()

blank_dir = r'C:\Users\Habram\Documents\Datasets\fake-invoices\blank.tif'
blank = Image.open(blank_dir)

invoice_generator = InvoiceGenerator(blank, 1, True)
invoice_generator()

C:\Users\Habram\AppData\Local\Temp\ipykernel_4344\1540989827.py:169: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = self.draw.textsize(summa + ' ' + self.currency, font=self.font)


In [ ]:
def text_label(draw, coords, text, font, anchor, label):
    '''
    Draws text on a document, returns its bounding box
    Whole text-level bounding box + word-level bounding boxes are obtained
    Prints text left to right (lm) or right to left (rm) based on the 'anchor' parameter
    
    Args:
        draw        -- Pillow draw object
        coords      -- upper left coordinate of the text to be drawn
        text        -- text to be drawn
        font        -- Pillow font object
        anchor      -- Anchor position of the text (default: lt = left top)
        label       -- Label class of the text
    '''
    # Create dictionary which contains all of the label information
    text_labels = {
        'text': text,
        'label': label,
        'box': None,
        'words': []
    }

    # If the text is positioned to the left, just shift the X position, and position everything to the right
    if anchor == 'rm':
        anchor = 'lm'
        text_box = draw.textbbox(coords, text, font=font, anchor=anchor)
        text_width = text_box[2] - text_box[0]
        x1 = coords[0] - text_width
        y1 = coords[1]
        coords = [x1, y1]

    # Get the top left corner of the text
    x1, y1 = coords

    # Get the width of a space caracter
    space_bbox = draw.textbbox(coords, ' ', font=font, anchor=anchor)
    space_width = space_bbox[2] - space_bbox[0]

    # Initialize the text bounding box
    text_x1, text_y1, text_x2, text_y2 = draw.textbbox(coords, text, font=font, anchor=anchor)
    
    # Get the individual words
    words = text.split()
    for word in words:
        # Draw the word
        draw.text([x1, y1], word, fill='black', font=font, anchor=anchor)

        # Width of the word
        word_bbox = draw.textbbox([x1, y1], word, font=font, anchor=anchor)
        word_width = word_bbox[2] - word_bbox[0]
        draw.rectangle(word_bbox, outline='red')

        # Add the individual word to the dictionary
        text_labels['words'].append(
            {
            'box':  word_bbox,
            'text': word
            }
        )

        # Slide the x position with the word width and a space
        x1 += word_width + space_width

        # Update the whole text bounding box
        if word_bbox[2] > text_x2: text_x2 = word_bbox[2]
        if word_bbox[3] > text_y2: text_y2 = word_bbox[3]

    draw.rectangle([text_x1, text_y1, text_x2, text_y2], outline='green')
    text_labels['box'] = [text_x1, text_y1, text_x2, text_y2]

    return text_labels

blank_dir = r'C:\Users\Habram\Documents\Datasets\fake-invoices\blank.tif'
blank = Image.open(blank_dir)
draw = ImageDraw.Draw(blank)
font = ImageFont.truetype("arial.ttf", 30)

label = text_label(draw, [800, 100], 'Where are those happy days?', font, anchor='lm', label='other')
label = text_label(draw, [800, 100], 'Where are those happy days?', font, anchor='rm', label='other')
print(label)
blank.show()

In [ ]:
import json

with open('annotation.json', 'w') as fp:
    json.dump(label, fp)